<a href="https://colab.research.google.com/github/liguoqiu/TaskProject/blob/master/begin1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip3 install https://download.pytorch.org/whl/cu80/torch-1.0.0-cp36-cp36m-linux_x86_64.whl
!pip3 install torchvision

     |████████████████████████████████| 532.5MB 32kB/s 
ERROR: torchvision 0.4.1+cu100 has requirement torch==1.3.0, but you'll have torch 1.0.0 which is incompatible.
  Found existing installation: torch 1.3.0+cu100
    Uninstalling torch-1.3.0+cu100:
      Successfully uninstalled torch-1.3.0+cu100
     |████████████████████████████████| 773.1MB 24kB/s 
  Found existing installation: torch 1.0.0
    Uninstalling torch-1.0.0:
      Successfully uninstalled torch-1.0.0


In [3]:
import torch
x=torch.empty(5,3)
print(x)
 



tensor([[2.0698e-36, 0.0000e+00, 0.0000e+00],
        [0.0000e+00, 0.0000e+00, 0.0000e+00],
        [0.0000e+00, 0.0000e+00, 2.8026e-45],
        [0.0000e+00, 1.1210e-44, 0.0000e+00],
        [1.4013e-45, 0.0000e+00, 1.1939e-39]])


In [4]:
!/opt/bin/nvidia-smi

Mon Nov  4 05:52:06 2019       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 418.67       Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   31C    P8    27W / 149W |      0MiB / 11441MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

In [0]:
import torch
from torch import nn,optim
from torch.autograd import Variable
from torch.utils.data import DataLoader
from torchvision import datasets,transforms


batch_size=64
learning_rate=0.01
num_epoches=20

data_tf=transforms.Compose([transforms.ToTensor(),transforms.Normalize([0.5],[0.5])])


In [3]:
train_dataset=datasets.MNIST(root='./data',train=True,transform=data_tf,download=True)
test_dataset=datasets.MNIST(root='./data',train=False,transform=data_tf)
train_loader=DataLoader(train_dataset,batch_size=batch_size,shuffle=True)
test_loader=DataLoader(test_dataset,batch_size=batch_size,shuffle=False)

  0%|          | 0/9912422 [00:00<?, ?it/s]

9920512it [00:01, 8966834.10it/s]                             


Extracting ./data/MNIST/raw/train-images-idx3-ubyte.gz to ./data/MNIST/raw


  0%|          | 0/28881 [00:00<?, ?it/s]

32768it [00:00, 129311.96it/s]           
  0%|          | 0/1648877 [00:00<?, ?it/s]

Extracting ./data/MNIST/raw/train-labels-idx1-ubyte.gz to ./data/MNIST/raw


1654784it [00:00, 2345409.51it/s]                           
0it [00:00, ?it/s]

Extracting ./data/MNIST/raw/t10k-images-idx3-ubyte.gz to ./data/MNIST/raw


8192it [00:00, 52859.10it/s]            

Extracting ./data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ./data/MNIST/raw
Processing...
Done!


In [0]:
class Batch_Net(nn.Module):
  def __init__(self,in_dim,hidden1,hidden2,out_dim):
    super(Batch_Net,self).__init__()
    self.layer1=nn.Sequential(nn.Linear(in_dim,hidden1),nn.BatchNorm1d(hidden1),nn.ReLU(True))
    self.layer2=nn.Sequential(nn.Linear(hidden1,hidden2),nn.BatchNorm1d(hidden2),nn.ReLU(True))
    self.layer3=nn.Sequential(nn.Linear(hidden2,out_dim))
  def forward(self,x):
    x=self.layer1(x)
    x=self.layer2(x)
    x=self.layer3(x)
    return x


In [0]:
model=Batch_Net(28*28,300,100,10)
model=model.cuda()
criterion=nn.CrossEntropyLoss()
optimizer=optim.SGD(model.parameters(),lr=learning_rate)


In [6]:
model.eval()
eval_loss=0
eval_acc=0
for data in test_loader:
  img,label=data
  img=img.view(img.size(0),-1)
  with torch.no_grad():
    img=Variable(img).cuda()
    label=Variable(label).cuda()
  out=model(img)
  loss=criterion(out,label)
  eval_loss+=loss.item()*label.size(0)
  _,pred=torch.max(out,1)
  num_correct=(pred==label).sum()
  eval_acc+=num_correct.item()
  print('TestLoss:{:.6f},ACC:{:.6f}'.format(eval_loss/(len(test_dataset)),eval_acc/(len(test_dataset))))


TestLoss:0.014770,ACC:0.000600
TestLoss:0.029329,ACC:0.001400
TestLoss:0.044429,ACC:0.001900
TestLoss:0.059214,ACC:0.002500
TestLoss:0.074170,ACC:0.002900
TestLoss:0.089176,ACC:0.003600
TestLoss:0.104067,ACC:0.004500
TestLoss:0.119053,ACC:0.005100
TestLoss:0.133790,ACC:0.005600
TestLoss:0.148631,ACC:0.006600
TestLoss:0.163543,ACC:0.007400
TestLoss:0.178323,ACC:0.007800
TestLoss:0.193199,ACC:0.008500
TestLoss:0.207985,ACC:0.009000
TestLoss:0.222679,ACC:0.009300
TestLoss:0.237639,ACC:0.009900
TestLoss:0.252540,ACC:0.010800
TestLoss:0.267400,ACC:0.011500
TestLoss:0.282225,ACC:0.011900
TestLoss:0.297083,ACC:0.012800
TestLoss:0.312085,ACC:0.013000
TestLoss:0.326832,ACC:0.014400
TestLoss:0.341781,ACC:0.015000
TestLoss:0.356579,ACC:0.016000
TestLoss:0.371364,ACC:0.016800
TestLoss:0.386203,ACC:0.017400
TestLoss:0.401107,ACC:0.018000
TestLoss:0.416042,ACC:0.018400
TestLoss:0.430935,ACC:0.019000
TestLoss:0.445945,ACC:0.019700
TestLoss:0.460609,ACC:0.020400
TestLoss:0.475523,ACC:0.020700
TestLoss

In [14]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class Net(nn.Module):

    def __init__(self):
        super(Net, self).__init__()
        # 1 input image channel, 6 output channels, 5x5 square convolution
        # kernel
        self.conv1 = nn.Conv2d(1, 6, 5)
        self.conv2 = nn.Conv2d(6, 16, 5)
        # an affine operation: y = Wx + b
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        # Max pooling over a (2, 2) window
        x = F.max_pool2d(F.relu(self.conv1(x)), (2, 2))
        # If the size is a square you can only specify a single number
        x = F.max_pool2d(F.relu(self.conv2(x)), 2)
        x = x.view(-1, self.num_flat_features(x))
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

    def num_flat_features(self, x):
        size = x.size()[1:]  # all dimensions except the batch dimension
        num_features = 1
        for s in size:
            num_features *= s
        return num_features


net = Net()
print(net)


Net(
  (conv1): Conv2d(1, 6, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=400, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)


In [18]:
import torch
import torchvision
import torchvision.transforms as transforms
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=4,
                                          shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=4,
                                         shuffle=False, num_workers=2)

classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

0it [00:00, ?it/s]

170500096it [00:04, 39591706.37it/s]                               


Extracting ./data/cifar-10-python.tar.gz to ./data
Files already downloaded and verified


In [0]:
import torch.nn as nn
import torch.nn.functional as F

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 16 * 5 * 5)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

net = Net()



In [0]:
import torch.optim as optim
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

In [21]:
for epoch in range(2):  # loop over the dataset multiple times

    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        # get the inputs
        inputs, labels = data

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        if i % 2000 == 1999:    # print every 2000 mini-batches
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 2000))
            running_loss = 0.0

print('Finished Training')

[1,  2000] loss: 2.198
[1,  4000] loss: 1.862
[1,  6000] loss: 1.677
[1,  8000] loss: 1.577
[1, 10000] loss: 1.512
[1, 12000] loss: 1.472
[2,  2000] loss: 1.386
[2,  4000] loss: 1.357
[2,  6000] loss: 1.346
[2,  8000] loss: 1.312
[2, 10000] loss: 1.304
[2, 12000] loss: 1.261
Finished Training


In [22]:
dataiter = iter(testloader)
images, labels = dataiter.next()
# print images
imshow(torchvision.utils.make_grid(images))
print('GroundTruth: ', ' '.join('%5s' % classes[labels[j]] for j in range(4)))

NameError: ignored

In [23]:
correct = 0
total = 0
with torch.no_grad():
    for data in testloader:
        images, labels = data
        outputs = net(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print('Accuracy of the network on the 10000 test images: %d %%' % (
    100 * correct / total))

Accuracy of the network on the 10000 test images: 55 %


In [6]:
import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=4,
                                          shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=4,
                                         shuffle=False, num_workers=2)

classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

Files already downloaded and verified
Files already downloaded and verified


In [9]:
class Net(nn.Module):
  def __init__(self):
    super(Net, self).__init__()
    self.conv1 = nn.Conv2d(3, 14, 5)
    self.pool = nn.MaxPool2d(2, 2)
    self.conv2 = nn.Conv2d(14, 16, 5)
    self.fc1 = nn.Linear(16 * 5 * 5, 120)
    self.fc2 = nn.Linear(120, 84)
    self.fc3 = nn.Linear(84, 10)

  def forward(self, x):
    x = self.pool(F.relu(self.conv1(x)))
    x = self.pool(F.relu(self.conv2(x)))
    x = x.view(-1, 16 * 5 * 5)
    x = F.relu(self.fc1(x))
    x = F.relu(self.fc2(x))
    x = self.fc3(x)
    return x

net = Net()
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
net=net.to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

for epoch in range(2):
  run_loss=0.0
  for i,data in enumerate(trainloader,0):
    inputs,labels=data
    inputs, labels = inputs.to(device), labels.to(device)
    optimizer.zero_grad()
    outputs=net(inputs)
    loss=criterion(outputs,labels)
    loss.backward()
    optimizer.step()

    run_loss+=loss.item()
    if i%2000==1999:
      print('[%d,%5d] loss:%.3f'%(epoch,i+1,run_loss/2000))
      run_loss=0.0
print('finish')


    

[0, 2000] loss:2.214
[0, 4000] loss:1.831
[0, 6000] loss:1.635
[0, 8000] loss:1.542
[0,10000] loss:1.489
[0,12000] loss:1.415
[1, 2000] loss:1.348
[1, 4000] loss:1.314
[1, 6000] loss:1.281
[1, 8000] loss:1.264
[1,10000] loss:1.235
[1,12000] loss:1.216
finish
